In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import datetime as dt

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# We can view all of the classes that automap found
Base.classes.keys()

In [ ]:
['measurement', 'station']

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results
max_date= session.query(func.max(Measurement.date)).all()
print(max_date)

# Calculate the date 1 year ago from the last data point in the database
prev_yr = dt.date(2017,8,23)-dt.timedelta(days=365)
prev_yr

# Perform a query to retrieve the data and precipitation scores
results = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date>=prev_yr).all()

# Save the query results as a Pandas DataFrame and set the index to the date column
df = pd.DataFrame(results,columns=['Date','Precipitation'])
df.set_index(df['Date'], inplace=True)


# Sort the dataframe by date
df = df.sort_index()

# Use Pandas Plotting with Matplotlib to plot the data
#currently showing how to plot the same onformation two ways
one = df.plot(figsize=(10,5))
two = df.plot(rot=90)
two

In [ ]:
[('2017-08-23',)]

In [ ]:
<matplotlib.axes._subplots.AxesSubplot at 0x2416dd23e80>

In [ ]:
# Design a query to show how many stations are available in this dataset?
data = pd.read_csv("Resources/hawaii_stations.csv")
data.head(5)

In [ ]:
# Design a query to show how many stations are available in this dataset?
stations_count = len(data['station'].unique())
stations_count

In [ ]:
9

In [ ]:
session.query(Measurement.station, func.count(Measurement.station)).\
group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()

In [ ]:
[('USC00519281', 2772),
 ('USC00519397', 2724),
 ('USC00513117', 2709),
 ('USC00519523', 2669),
 ('USC00516128', 2612),
 ('USC00514830', 2202),
 ('USC00511918', 1979),
 ('USC00517948', 1372),
 ('USC00518838', 511)]

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature most active station?
session.query(func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs)).\
        filter(Measurement.station == 'USC00519281').all()

In [ ]:
[(54.0, 85.0, 71.66378066378067)]

In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
all_temps = session.query(Measurement.tobs).\
        filter(Measurement.date >= prev_yr).filter(Measurement.station == 'USC00519281').all()

In [ ]:
df = pd.DataFrame(all_temps,columns=['tobs'])
df.plot.hist()

In [ ]:
<matplotlib.axes._subplots.AxesSubplot at 0x2416edb8c88>

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
[(62.0, 69.57142857142857, 74.0)]

In [ ]:
:start_date = dt.date(2017,8,20)-dt.timedelta(days=365)
end_date = dt.date(2017,8,26)-dt.timedelta(days=365)

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.
t_min,t_avg,t_max = calc_temps(start_date.strftime("%Y-%m-%d"),end_date.strftime("%Y-%m-%d"))[0]
print(t_min,t_avg,t_max)

In [ ]:
72.0 78.53333333333333 84.0

In [ ]:
<IPython.core.display.Javascript object>

In [ ]:
rainfall = session.query(Station.station, Station.name,Station.latitude, Station.longitude, Station.elevation,func.sum(Measurement.prcp)).\
        filter(Measurement.date >= start_date).filter(Measurement.date >= end_date).group_by(Station.name).order_by(func.sum(Measurement.prcp).desc()).all()
print(rainfall)

In [ ]:
[('USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5, 345.7000000000006), ('USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0, 345.7000000000006), ('USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999998, 32.9, 345.7000000000006), ('USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6, 345.7000000000006), ('USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9, 345.7000000000006), ('USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4, 345.7000000000006), ('USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0, 345.7000000000006), ('USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6, 345.7000000000006), ('USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9, 345.7000000000006)]